# Use Decision Optimization to plan your diet with `ibm-watson-machine-learning`

This notebook facilitates Decision Optimization and Watson Machine Learning services. It contains steps and code to work with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. It also introduces commands for getting model and training data, persisting model, deploying model and scoring it.

Some familiarity with Python is helpful. This notebook uses Python 3.8.

## Learning goals

The learning goals of this notebook are:

-  Load a DO model file into an Watson Machine learning repository.
-  Prepare data for training and evaluation.
-  Create an DO machine learning job.
-  Persist a DO model Watson Machine Learning repository.
-  Deploy a model for batch scoring using Wastson Machine Learning API.

## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.  [Download externally created DO model](#download)
3.	[Persist externally created DO model](#persistence)
4.	[Deploy in a Cloud](#deploy)
5.	[Create job](#job)
6.  [Clean up](#cleanup)
7.	[Summary and next steps](#summary)



<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [1]:
api_key = 'XXX'
location = 'us-south'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [3]:
!pip install -U ibm-watson-machine-learning
!pip install -U wget

     |████████████████████████████████| 1.9 MB 20.9 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.118
    Uninstalling ibm-watson-machine-learning-1.0.118:
      Successfully uninstalled ibm-watson-machine-learning-1.0.118
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=32939ce2c9d1055f5bb26380fab3d9db7d936b1b3ab929b9328a6e309f9ea842
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [4]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = 'XXX'

You can use `list` method to print all existing spaces.

In [6]:
client.spaces.list(limit=10)

------------------------------------  ------------------------  ------------------------
ID                                    NAME                      CREATED
b4120a07-a352-4a16-b53d-8e480372c585  Orchestration Flow Space  2021-07-29T14:28:08.752Z
a3885cea-911c-4cb6-98fb-4e37874c3d56  Shelf Space Optimization  2021-07-27T22:00:48.617Z
647c4c2b-a83e-447f-b617-462806bf1ba5  IntelligentMaintenance    2021-07-21T22:09:46.668Z
0463854a-56af-412a-b0e7-33463ca2e6a4  Network Design DS         2021-06-23T17:52:35.238Z
47c5c0ba-95cf-4683-9d35-54e2d58599f8  My Deployment Space       2021-06-22T16:07:42.682Z
------------------------------------  ------------------------  ------------------------


To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [7]:
client.set.default_space(space_id)

'SUCCESS'

#### <a id="download"></a>
## 2. Download externally created Keras model and data


In this section, you will download externally created DO models and data used for training it.


**Action**: Get your DO model.

In [8]:
import os
import wget
model_path = 'do-model.tar.gz'
if not os.path.isfile(model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/models/decision_optimization/do-model.tar.gz")


<a id="persistence"></a>
## 3. Persist externally created DO model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.


### 3.1: Publish model


#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.


Get software specification for DO model

In [9]:
sofware_spec_uid = client.software_specifications.get_uid_by_name("do_12.9")

Output data schema for storing model in WML repository

In [10]:
output_data_schema = [{'id': 'stest',
                              'type': 'list',
                              'fields': [{'name': 'age', 'type': 'float'},
                                         {'name': 'sex', 'type': 'float'},
                                         {'name': 'cp', 'type': 'float'},
                                         {'name': 'restbp', 'type': 'float'},
                                         {'name': 'chol', 'type': 'float'},
                                         {'name': 'fbs', 'type': 'float'},
                                         {'name': 'restecg', 'type': 'float'},
                                         {'name': 'thalach', 'type': 'float'},
                                         {'name': 'exang', 'type': 'float'},
                                         {'name': 'oldpeak', 'type': 'float'},
                                         {'name': 'slope', 'type': 'float'},
                                         {'name': 'ca', 'type': 'float'},
                                         {'name': 'thal', 'type': 'float'}]
                               }, {'id': 'teste2',
                 'type': 'test',
                 'fields': [{'name': 'age', 'type': 'float'},
                            {'name': 'sex', 'type': 'float'},
                            {'name': 'cp', 'type': 'float'},
                            {'name': 'restbp', 'type': 'float'},
                            {'name': 'chol', 'type': 'float'},
                            {'name': 'fbs', 'type': 'float'},
                            {'name': 'restecg', 'type': 'float'},
                            {'name': 'thalach', 'type': 'float'},
                            {'name': 'exang', 'type': 'float'},
                            {'name': 'oldpeak', 'type': 'float'},
                            {'name': 'slope', 'type': 'float'},
                            {'name': 'ca', 'type': 'float'},
                            {'name': 'thal', 'type': 'float'}]}]

In [11]:
model_meta_props = {
                        client.repository.ModelMetaNames.NAME: "LOCALLY created DO model",
                        client.repository.ModelMetaNames.TYPE: "do-docplex_12.9",
                        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid,
                        client.repository.ModelMetaNames.OUTPUT_DATA_SCHEMA: output_data_schema
                    }
published_model = client.repository.store_model(model=model_path, meta_props=model_meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

### 3.2: Get model details


In [12]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "schemas": {
      "input": [],
      "output": [
        {
          "fields": [
            {
              "name": "age",
              "type": "float"
            },
            {
              "name": "sex",
              "type": "float"
            },
            {
              "name": "cp",
              "type": "float"
            },
            {
              "name": "restbp",
              "type": "float"
            },
            {
              "name": "chol",
              "type": "float"
            },
            {
              "name": "fbs",
              "type": "float"
            },
            {
              "name": "restecg",
              "type": "float"
            },
            {
              "name": "thalach",
              "type": "float"
            },
            {
              "name": "exang",
              "type": "float"
            },
            {
              "name": "oldpeak",
              "type": "float"
            },
 

### 3.3 Get all models

In [ ]:
client.repository.list_models()

<a id="deploy"></a>
## 4. Deploy in a Cloud
In this section you will learn how to create batch deployment to create job using the Watson Machine Learning Client.

You can use commands bellow to create batch deployment for stored model (web service).


### 4.1: Create model deployment


In [14]:
meta_data = {
    client.deployments.ConfigurationMetaNames.NAME: "deployment_DO",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S", "num_nodes": 1}

}
deployment_details = client.deployments.create(published_model_uid, meta_props=meta_data)



#######################################################################################

Synchronous deployment creation for uid: 'cf9c0eb0-9188-4dfc-b06e-46277684015f' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b5396889-f290-47ac-b45d-8da12c3ad69f'
------------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.


In [15]:
deployment_uid = client.deployments.get_uid(deployment_details)

Now, You can list all deployments.

In [ ]:
client.deployments.list()

### 4.2: Get deployment details


In [ ]:
client.deployments.get_details(deployment_uid)

<a id="job"></a>
## 5. Create job

You can create job to web-service deployment using `create_job` method.

Prepare test data 

In [18]:
import pandas as pd
diet_food = pd.DataFrame([["Roasted Chicken", 0.84, 0, 10],
                          ["Spaghetti W/ Sauce", 0.78, 0, 10],
                          ["Tomato,Red,Ripe,Raw", 0.27, 0, 10],
                          ["Apple,Raw,W/Skin", 0.24, 0, 10],
                          ["Grapes", 0.32, 0, 10],
                          ["Chocolate Chip Cookies", 0.03, 0, 10],
                          ["Lowfat Milk", 0.23, 0, 10],
                          ["Raisin Brn", 0.34, 0, 10],
                          ["Hotdog", 0.31, 0, 10]], columns=["name", "unit_cost", "qmin", "qmax"])

diet_food_nutrients = pd.DataFrame([
    ["Spaghetti W/ Sauce", 358.2, 80.2, 2.3, 3055.2, 11.6, 58.3, 8.2],
    ["Roasted Chicken", 277.4, 21.9, 1.8, 77.4, 0, 0, 42.2],
    ["Tomato,Red,Ripe,Raw", 25.8, 6.2, 0.6, 766.3, 1.4, 5.7, 1],
    ["Apple,Raw,W/Skin", 81.4, 9.7, 0.2, 73.1, 3.7, 21, 0.3],
    ["Grapes", 15.1, 3.4, 0.1, 24, 0.2, 4.1, 0.2],
    ["Chocolate Chip Cookies", 78.1, 6.2, 0.4, 101.8, 0, 9.3, 0.9],
    ["Lowfat Milk", 121.2, 296.7, 0.1, 500.2, 0, 11.7, 8.1],
    ["Raisin Brn", 115.1, 12.9, 16.8, 1250.2, 4, 27.9, 4],
    ["Hotdog", 242.1, 23.5, 2.3, 0, 0, 18, 10.4]
], columns=["Food", "Calories", "Calcium", "Iron", "Vit_A", "Dietary_Fiber", "Carbohydrates", "Protein"])

diet_nutrients = pd.DataFrame([
    ["Calories", 2000, 2500],
    ["Calcium", 800, 1600],
    ["Iron", 10, 30],
    ["Vit_A", 5000, 50000],
    ["Dietary_Fiber", 25, 100],
    ["Carbohydrates", 0, 300],
    ["Protein", 50, 100]
], columns=["name", "qmin", "qmax"])

In [19]:
job_payload_ref = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [
        {
            "id": "diet_food.csv",
            "values": diet_food
        },
        {
            "id": "diet_food_nutrients.csv",
            "values": diet_food_nutrients
        },
        {
            "id": "diet_nutrients.csv",
            "values": diet_nutrients
        }
    ],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [
        {
            "id": ".*.csv"
        }
    ]
}

Create job using Watson Machine Learning client

In [20]:
job = client.deployments.create_job(deployment_uid, meta_props=job_payload_ref)

Checking created job status and calculated KPI.

In [21]:
import time

job_id = client.deployments.get_job_uid(job)

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    job_details_do = client.deployments.get_job_details(job_id)
    kpi = job_details_do['entity']['decision_optimization']['solve_state']['details']['KPI.Total Calories']
    print(f"KPI: {kpi}")
else:
    print("Job hasn't completed successfully in 5 minutes.")

KPI: 2000.0


<a id="cleanup"></a>
## 6. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 7. Summary and next steps

You successfully completed this notebook! You learned how to use Keras machine learning library as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=analytics?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 


### Authors

**Wojciech Jargielo**, Software Engineer

Copyright © 2020, 2021 IBM. This notebook and its source code are released under the terms of the MIT License.
